# DataLab Cup 1: Predicting Appropriate Response

## Team name: 卍閃傲o剎帝卍
### members:
- 106024510 陳俊穎
- 106024513 曾奕齊
- 106024519 吳亦振
- 106024509 曾立豪

## 1. Cut word via Jieba

使用結巴中文分詞，模式使用全模式，把句中所有的可以成詞的詞語都掃描出來。

In [ ]:
%matplotlib inline

# 套件
import math
import time
import os
os.chdir('C:/Users/a0972/Desktop/Deep Learning/Competitions')
import pandas as pd
import numpy as np

# 讀檔
NUM_PROGRAM = 8
programs = []
for i in range(1, NUM_PROGRAM+1):
    program = pd.read_csv('Program0%d.csv' % (i))
    
    print('Program %d' % (i))
    print('Episodes: %d' % (len(program)))
    print(program.columns)
    print()
    
    print(program.loc[:1]['Content'])
    print()
    
    programs.append(program)
    
questions = pd.read_csv('Question.csv')
print('Question')
print('Episodes: %d' % (len(questions)))
print(questions.columns)
print()
print(questions.loc[:2]['Question'])
print()

for i in range(6):
    print(questions.loc[:2]['Option%d' % (i)])
    print()
    
# 結巴  
import jieba

jieba.set_dictionary('big5_dict.txt')
example_str = '我討厭吃蘋果'
cut_example_str = jieba.lcut(example_str)
print(cut_example_str)


def jieba_lines(lines): # 將複數行分詞
    cut_lines = []
    
    for line in lines:
        cut_line = jieba.lcut(line, cut_all=True) # 全模式
        cut_lines.append(cut_line)
    
    return cut_lines

# 分 program
cut_programs = []
for program in programs:
    episodes = len(program)
    cut_program = []
    
    for episode in range(episodes):
        lines = program.loc[episode]['Content'].split('\n')
        cut_program.append(jieba_lines(lines))
    
    cut_programs.append(cut_program)

print("%d programs" % len(cut_programs))
print("%d episodes in program 0" % len(cut_programs[0]))
print("%d lines in first episode of program 0" % len(cut_programs[0][0]))

print()
print("first 3 lines in 1st episode of program 0: ")
print(cut_programs[0][0][:3])

# 分 question
cut_questions = []
n = len(questions)
for i in range(n):
    cut_question = []
    lines = questions.loc[i]['Question'].split('\n')
    cut_question.append(jieba_lines(lines))
    
    for j in range(6):
        line = questions.loc[i]['Option%d' % (j)]
        cut_question.append(jieba.lcut(line, cut_all=True))
    
    cut_questions.append(cut_question)
    
    
print("%d questions" % len(cut_questions))
print(len(cut_questions[0]))

# 1 question
print(cut_questions[0][0])

# 6 optional reponses
for i in range(1, 7):
    print(cut_questions[0][i])
    
# 存檔
import numpy as np

  

## 2. Data preprocessing

為了使詞庫更加乾淨，我們刪除空白、標點符號以及空字串。

In [ ]:
def delete_blank(x): # 空白、標點符號
    bad = []
    for i in range(0, len(x)) :
        if x[i] == ' ' or x[i] == ',' or x[i] == '...' or x[i] == '' :
            bad.append(i)
    kkk = np.array(bad) - np.array(range(0, len(bad)))
    for j in kkk:
        del x[j]
        
    return x

def delete_noword(x): # 空字串
    bad = []
    for i in range(0, len(x)):
        if len(x[i]) == 0:
            bad.append(i)
    kkk = np.array(bad) - np.array(range(0, len(bad)))
    for j in kkk :
        del x[j]
    return x

# 處理 program    
for i in range(0, 8):
    for j in range(0, len(cut_programs[i])) :
        delete_noword(cut_programs[i][j])

for i in range(0, 8):
    for j in range(0, len(cut_programs[i])) :
        for k in range(0, len(cut_programs[i][j])):
            cut_programs[i][j][k] = delete_blank(cut_programs[i][j][k]) 
            
# 處理 question
for i in range(0, 500):
    cut_Question[i][0] = delete_blank(list(chain(*cut_Question[i][0])))
    for j in range(1, 7):
        cut_Question[i][j] = delete_blank(cut_Question[i][j])



## 3. Word2Vec

利用 gensim 套件訓練 programs 的詞，轉換成200維的向量，再從 programs 裡刪除不在詞庫的詞。

參數設定：
- size=200 
- window=10
- min_count=50
- workers=4 
- sg=1

In [ ]:
# 把詞整理好
w2v_text = []
for i in range(0, 8) :      
    for j in range(0, len(cut_programs[i])) :
       w2v_text = w2v_text + [list(chain(*cut_programs[i][j]))]


# Word2Vec
from gensim.models.word2vec import Word2Vec

# 訓練
start = time.time()        
        
model = Word2Vec(w2v_text,size=200, window=10, min_count=50, workers=4, sg=1)

end = time.time()
print("Running Time : %d min %d sec"%(math.floor((end - start) / 60),math.ceil((end - start) % 60 )))


def in_dict(text, d): # 是否在字典中
    return [x for x in text if x in d]

# 刪除沒有train到的詞
start = time.time()

for i in range(0, 8):
    for j in range(0, len(cut_programs[i])) :
        for k in range(0, len(cut_programs[i][j])):
            cut_programs[i][j][k] = in_dict(cut_programs[i][j][k], list(model.wv.vocab.keys()))

for i in range(0, 500):
    cut_Question[i][0] = in_dict(cut_Question[i][0], list(model.wv.vocab.keys()))
    for j in range(1, 7):
        cut_Question[i][j] = in_dict(cut_Question[i][j], list(model.wv.vocab.keys()))

end = time.time()

print("Running Time : %d min %d sec"%(math.floor((end - start) / 60),math.ceil((end - start) % 60 )))


# 將不在詞庫的詞捨棄
for i in range(0, 8):
    for j in range(0, len(cut_programs[i])) :
        delete_noword(cut_programs[i][j])

for i in range(0, 8):
    for j in range(0, len(cut_programs[i])) :
        for k in range(0, len(cut_programs[i][j])):
            cut_programs[i][j][k] = delete_blank(cut_programs[i][j][k]) 
            

for i in range(0, 500):
    for j in range(0, 7):
        cut_Question[i][j] = delete_blank(cut_Question[i][j])

## 4. Training set generation

生成300000筆資料當作 training data ，每筆資料包含兩個句子，其中約莫150000筆的句子具有前後文關係，其餘則無。
接著將句中單詞使用 word2vec 轉換成200維的向量，假設前文與後文分別有由單詞轉換而成的 n 個向量$v_1,...,v_n$與 m 個向量$u_1,...,u_m$，我們設計模型特徵如下：

 1. average vector 1 : $Vmean=(v_1+\cdots +v_n)/n$ 


 2. average vector 2 : $Umean=(u_1+\cdots +u_m)/m$
 
 
 3. correlation of average vector 1 and 2 : $Corr=corr(Vmean, Umean)$


 4. average maximum correlation every words : $meanmaxCorr = (c_1+\cdots +c_n)/n, \quad where\quad c_i=\underset{j=1,...,m}{max}corr(v_i,u_j)$
 
 
 得到300000筆402維的training data。
 
 接著將 questions 中的 question 與每個 option 配對，以相同方式做特徵選取，得到500*6筆 testing data。

In [ ]:
import random

NUM_TRAIN = 300000
TRAIN_VALID_RATE = 0.7
NUM_PROGRAM = 8
def generate_training_data():
    Xs, Ys = [], []
    
    for i in range(NUM_TRAIN):
        pos_or_neg = random.randint(0, 1)
        
        if pos_or_neg==1:
            program_id = random.randint(0, NUM_PROGRAM-1)
            episode_id = random.randint(0, len(cut_programs[program_id])-1)
            line_id = random.randint(0, len(cut_programs[program_id][episode_id])-2)
            
            Xs.append([cut_programs[program_id][episode_id][line_id], 
                       cut_programs[program_id][episode_id][line_id+1]])
            Ys.append(1)
            
        else:
            first_program_id = random.randint(0, NUM_PROGRAM-1)
            first_episode_id = random.randint(0, len(cut_programs[first_program_id])-1)
            first_line_id = random.randint(0, len(cut_programs[first_program_id][first_episode_id])-1)
            
            second_program_id = random.randint(0, NUM_PROGRAM-1)
            second_episode_id = random.randint(0, len(cut_programs[second_program_id])-1)
            second_line_id = random.randint(0, len(cut_programs[second_program_id][second_episode_id])-1)
            
            Xs.append([cut_programs[first_program_id][first_episode_id][first_line_id], 
                       cut_programs[second_program_id][second_episode_id][second_line_id]])
            Ys.append(0)
    
    return Xs, Ys

Xs, Ys = generate_training_data()

# 特徵選取

train_X = []

for i in range(0, len(Xs)) :
    v0 = model[in_dict(Xs[i][0], list(model.wv.vocab.keys()))]
    v1 = model[in_dict(Xs[i][1], list(model.wv.vocab.keys()))]
    ss = []
    for a1 in range(0, v0.shape[0]):
        cor_in = []
        for a2 in range(0, v1.shape[0]):
            cor_in.append(np.absolute(np.corrcoef(v0[a1],v1[a2])[0][1]))
        ss.append(np.max(cor_in))
    cor = np.mean(ss)

    train_X.append(list(np.concatenate((model[Xs[i][0]].mean(axis = 0), model[Xs[i][1]].mean(axis = 0), np.array([abs(cor)])))))
    
train_X = np.array(train_X)
y_train = np.array(Ys)


test_X = []

for i in range(0, len(cut_Question)):
    if len(in_dict(cut_Question[i][0], list(model.wv.vocab.keys()))) == 0:
        pass
    else :
        v0 = model[in_dict(cut_Question[i][0], list(model.wv.vocab.keys()))]
        Q = np.mean(v0, axis = 0)
        
        for j in range(1, 7):
            
            if len(in_dict(cut_Question[i][j], list(model.wv.vocab.keys()))) == 0:
                
                cor = 0
                
            else:
                v1 = model[in_dict(cut_Question[i][j], list(model.wv.vocab.keys()))]
                A = np.mean(v1, axis = 0)
                ss = []
                for a1 in range(0, v0.shape[0]):
                    cor_in = []
                    for a2 in range(0, v1.shape[0]):
                        cor_in.append(np.absolute(np.corrcoef(v0[a1],v1[a2])[0][1]))
                    ss.append(np.max(cor_in))
                cor = np.mean(ss)
            
            test_X.append(list(np.concatenate((Q, A, np.array([abs(cor)])))))   

test_X = np.array(test_X)

## 5. Build Model - LightGBM

用 LightGBM 訓練模型並做 5-fold CV，其參數以過往經驗微調。

LightGBM 是一個梯度 boosting 框架，使用基於學習算法的決策樹。它可以說是分布式的，高效的，根據官方文檔以及無數人的實際體驗，它有以下優勢：


- 更快的訓練效率

- 低內存使用

- 更好的準確率

- 支持並行學習，可處理大規模數據

在公開數據上的實驗表明 LightGBM 能在學習效率和準確率上都表現出比其他已有 boosting 工具更好的表現，而且有著更低的內存消耗。此外，實驗也表明 LightGBM 通過使用多台機器進行特定設定的訓練，它能取得線性加速。

In [ ]:
from sklearn.metrics import mean_absolute_error
from sklearn.cross_validation import KFold
import lightgbm as lgb 
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

train = pd.DataFrame(train_X)
tauc = np.array(y_train)
test = pd.DataFrame(test_X)

folds = 5
fpred2 = []
lgbclf = []

kf = KFold(train.shape[0], n_folds=folds)
for i, (train_index, test_index) in enumerate(kf):
    print('\n Fold %d\n' % (i + 1))
    X_train, X_val = train.iloc[train_index,:], train.iloc[test_index,:]
    y_train2, y_val2 = tauc[train_index], tauc[test_index]
    
    # Define cross-validation variables  
    param2={
    'boosting_type': 'gbdt',
    'class_weight': None,
    'colsample_bytree': 0.733333,
    'learning_rate': 0.0764107,
    'max_depth': -1,
    'min_child_samples': 460,
    'min_child_weight': 0.001,
    'min_split_gain': 0.0,
    'n_estimators': 7000,
    'n_jobs': -1,
    'num_leaves': 20,
    'objective':'binary',
    'random_state': 42,
    'reg_alpha': 0.877551,
    'reg_lambda': 0.204082,
    'silent': True,
    'subsample': 0.949495,
    'subsample_for_bin': 240000,
    'subsample_freq': 1,
    'metric': 'auc'
    } 
        
    train_data2 = lgb.Dataset(X_train,label=y_train2)
    valid_data2 = lgb.Dataset(X_val,label = y_val2)
    
    #  Build Model
    lgbm_clf = lgb.train(param2,
    train_data2,
    2500,
    valid_sets=valid_data2,
    early_stopping_rounds= 40,
    verbose_eval= 100
    )

    #  Evaluate Model and Predict  
    lgbclf  = lgbclf  + list(lgbm_clf.predict(X_val))
    y_pred2 = lgbm_clf.predict(test)

    #  feature importance
    feature_importances_clf = pd.DataFrame({'feature': train.columns, 'importance': lgbm_clf.feature_importance()})    
    zero_importance_features_clf = list(feature_importances_clf.feature[feature_importances_clf.importance == 0])

    #  Add Predictions and Average Them
    if i > 0:
        fpred2 = pred2 + y_pred2
        useless = list(set(useless) & set(zero_importance_features_clf) )
    else:
        fpred2 = y_pred2
        useless = zero_importance_features_clf
    pred2 = fpred2

zero_importance_features_clf_union = useless



## 6. Predict

對 Testing data 做預測，得出6個選項為前後文的機率後，各別乘上 $𝐶𝑜𝑟𝑟$ ，擇其最高作為6個選項中的預測結果。

In [ ]:
pred_clf = lgbclf
mpred2 = pred2 / folds
train = train.drop(labels = zero_importance_features_clf_union, axis = 1)
test = test.drop(labels = zero_importance_features_clf_union, axis = 1)

train_pred = np.where(np.array(pred_clf) >= 0.5,1,0)
test_pred = np.where(mpred2 >= 0.5,1,0)

sum(train_pred == tauc)/len(train_pred)

accuracy_score(train_pred, tauc)


yy = mpred2.reshape(500,6)
kk = (np.array(abs(test.iloc[:,400])) * mpred2).reshape(500,6)
      
prediction = np.argmax(kk,1)

submit = pd.read_csv('Sample.csv')
       
submit['Answer'] =  prediction


submit.to_csv('submit.csv', index = False)

## 7. Conclusions 

+ Public leaderboard：0.708   
+ Private leaderboard：0.656

由於平常較少處理文字資料的經驗，這次從文字的前處理到後續的建模的每一步都非常具有挑戰性，但過程非常有趣，我們四個人也從中學到非常多。在比賽的前期我們把重心放在問句與回覆之間correlation的計算，有趣的是透過word2vec為基礎的向量計算我們就達到了60%左右的Accuracy，比亂猜的18%高出不少，從實作中發現了較相關的字詞向量之間的確會有較高的correlation。然而在後期我們透過建machine learning的模型時卻落入了public leaderboard的陷阱，最後選的submission都是在public leaderboard表現比較好的，而不是在Cross validation準確率比較高的模型設定，導致沒有選到private leaderboard表現最好的那個結果(0.68)。從這點我們得到了一個寶貴的經驗，那就是對待validation set的結果應該要像對待老婆那樣，老婆絕對不會有錯，如果有錯那也一定是我們看錯。最後，真的很感謝有這次很棒的實作機會，也希望在未來的四個比賽我們能夠有相對更好的結果！